# Spin decomposition of operator 32 (part 2): Angular Mometum

## Imports

In [ ]:
from itertools import product

from pandas import read_csv, Series, DataFrame
import numpy as np
from sympy import S, Function
from numpwd.qchannels.cg import get_cg
from numpwd.integrate.angular import ReducedAngularPolynomial, get_x_mesh, get_phi_mesh
from numpwd.integrate.numeric import ExpressionMap

Read in results exported by part 1

In [ ]:
df = read_csv("data/operator-32-spin-pwd-lambda.csv").applymap(S)
df.head()

Specify mesh dimensions.
Because we know that this matrix element has at most angular momentum components of order `lmax = 2`, the required meshs for exact numerical integrations can be quite small.
If the denominator was present, the mesh must be larger to converge.

In [ ]:
lmax = 2
nx = 3
nphi = 7

# Value to decide when something is considered exactly zero
CUT = 1.0e-8

x, wx = get_x_mesh(nx)
phi, wphi = get_phi_mesh(nphi)
# The contracted Ylms
poly = ReducedAngularPolynomial(x, phi, lmax, wx=wx, wphi=wphi)

## Run integrations

In [ ]:
sachin_momentum_values = {"p_i": 100, "p_o": 200, "q_3": 300}
sachin_momentum_values

In [ ]:
def integrate(row):
    mla = row["m_lambda"]
    # get the sympy expression
    expr = row["val"]
    ee = expr.subs(sachin_momentum_values)
    # convert the operator expression to a tensor
    em = ExpressionMap(ee, ("x_o", "x_i", "phi"))
    # find all poly channels which have given mla and integrate against operator
    integrated_channels = poly.integrate(em(x, x, phi), mla)
    # give proper labels
    integrated_channels = Series(integrated_channels, name="val")
    integrated_channels.index.names = ("l_o", "l_i", "lambda", "m_lambda")
    integrated_channels = integrated_channels[np.abs(integrated_channels) > CUT]
    # check if imaginary part is zero (fails if not)
    assert all(np.abs(integrated_channels.values.imag) < CUT)
    # and return only real part
    return integrated_channels.apply(lambda el: el.real)

row = df.iloc[0]
print(row)
integrate(row)

Run integrations for all entries

In [ ]:
integrated = DataFrame(
    df.set_index(["ms_o_dm", "ms_i_dm", "s_o_nuc", "s_i_nuc", "ms_o_nuc", "ms_i_nuc"])
    .apply(integrate, axis=1)
    .stack(level=[0, 1, 2, 3])
).rename(columns={0: "res"})

integrated.index.rename(
    [n.replace("_nuc", "") for n in integrated.index.names], inplace=True
)
integrated

## Contract nuclear lsj channels

Finally, we have to project the reduced angular polynomial back to the $m_l$ components and use these to contract the $(ls)j$ system.
This is done by summing over $\lambda, m_\lambda$ and $m_l$ as well as $m_s$.

In [ ]:
CG = Function("CG")
fact = CG("l_o", "ml_o", "s_o", "ms_o", "j_o", "mj_o")
fact *= CG("l_i", "ml_i", "s_i", "ms_i", "j_i", "mj_i")
fact *= CG("l_i", "ml_i", "la", "m_la", "l_o", "ml_o")
fact

In [ ]:
def get_j_range(j1, j2):
    return range(abs(j1 - j2), j1 + j2 + 1)

def get_m_range(j):
    return range(-j, j + 1)

In [ ]:
def run_pwd(df):

    data = dict()
    # Sum over lambda and m_lambda
    for row in df.to_dict("records"):
        ranges = {
            "j_o": get_j_range(row["s_o"], row["l_o"]),
            "j_i": get_j_range(row["s_i"], row["l_i"]),
            "ml_o": get_m_range(row["l_o"]),
        }
        # sum over all m_s, m_l and collect by j m_j
        for vals in product(*ranges.values()):
            pars = dict(zip(ranges.keys(), vals))
            pars.update(row)
            pars["m_la"] = pars.pop("m_lambda")
            pars["la"] = pars.pop("lambda")
            pars["ml_i"] = pars["ml_o"] - pars["m_la"]
            pars["mj_i"] = pars["ml_i"] + pars["ms_i"]
            pars["mj_o"] = pars["ml_o"] + pars["ms_o"]
            if abs(pars["ml_i"]) > pars["l_i"]:
                continue
            if abs(pars["mj_i"]) > pars["j_i"]:
                continue
            if abs(pars["mj_o"]) > pars["j_o"]:
                continue

            key = (pars["j_o"], pars["j_i"], pars["mj_o"], pars["mj_i"])
            tmp = data.get(key, S(0))
            data[key] = tmp + float(fact.subs(pars).replace(CG, get_cg)) * row["res"]

    out = Series(data, name="val")
    out.index.names = ("j_o", "j_i", "mj_o", "mj_i")

    return out

group_keys = ["ms_o_dm", "ms_i_dm", "s_o", "s_i", "l_o", "l_i"]
res = DataFrame(integrated.reset_index().groupby(group_keys).agg(run_pwd)).query(
    "val > @CUT or val < -@CUT"
)
print("Found", res.size, "non-zero matrix elements")
res.head()

Select results with DM spin up and sort differently to simplify comparison with Sachin.

In [ ]:
dm_up = (
    res.reset_index()
    .query("ms_o_dm == ms_i_dm == 1/2")
    .set_index(
        ["l_o", "l_i", "s_o", "s_i", "j_o", "j_i", "mj_o", "mj_i", "ms_o_dm", "ms_i_dm"]
    )
    .sort_index()
    .reset_index()
)
dm_up.head(20)